# THE BURST

##  Database Creation and Upload


### Base dataset with cities and coordinates

In [ ]:
import requests
import pandas as pd
import xlrd

In [ ]:
df_cities=pd.read_csv('../raw_data/cities.csv')

In [ ]:
df_cities=df_cities[['code_commune_INSEE','code_postal','latitude','longitude','nom_commune_complet','code_departement','nom_departement','code_region','nom_region']]

In [ ]:
df_cities['coordinates'] = tuple(zip(df_cities.latitude,df_cities.longitude))

In [ ]:
df_cities=df_cities.drop_duplicates(subset=['code_commune_INSEE'])

In [ ]:
df_cities

### Load and transform airports and train stations databases

In [ ]:
df_airports=pd.read_excel('../raw_data/airports.xls', skiprows=[0,1])
w_header = df_airports.iloc[0] #grab the first row for the header #take the data less the header row
df_airports.columns = w_header #set the header row as the df header
df_airports = df_airports.drop(labels=0, axis=0)

In [ ]:
#We keep only the biggest airports
df_bigairports=df_airports[df_airports['Code IATA'].isin(['CDG','ORY','LYS','MRS','TLS','BSL','MLH','EAP','BOD','NTE','BVA','PTP','RUN','LIL','FDF','MPL','AJA','BIA','PPT','SXB','BES','BIQ','RNS','FSC','PUF','NOU','CAY','TLN','LDE','GEA','PGF','CFE'])]
df_bigairports.reset_index(inplace=True)

In [ ]:
df_bigairports.columns.values[4] = 'Latitude'
df_bigairports.columns.values[5] = 'Longitude'

In [ ]:
#We need to convert coordinates format
import re
def coord_converter(coordinates):
    list_=['Nord','Sud','Est','Ouest']
    if any(word in coordinates for word in list_):
        deg, minutes, seconds =  re.split('[°\']', coordinates)
        seconds,direction=seconds.split()
        return (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['Sud','Ouest'] else 1)
    else:
        deg, minutes, seconds =  re.split('[°\'"]', coordinates)
        return (float(deg) + float(minutes)/60 + float(seconds)/(60*60))

In [ ]:
df_bigairports.iloc[:,4] = df_bigairports.iloc[:,4].apply(lambda x: coord_converter(x))
df_bigairports.iloc[:,5] = df_bigairports.iloc[:,5].apply(lambda x: coord_converter(x))

In [ ]:
df_bigairports['coordinates'] = tuple(zip(df_bigairports.iloc[:,4],df_bigairports.iloc[:,5]))

In [ ]:
df_bigairports

In [ ]:
df_train=pd.read_excel('../raw_data/train.xls')

In [ ]:
df_train['coordinates'] = df_train['WGS 84'].apply(lambda x: tuple(map(str, x.split(', '))))

In [ ]:
lattrain=[]
lontrain=[]
for x in df_train.coordinates:
    lattrain.append(float(x[0]))
    lontrain.append(float(x[1]))
df_train['Latitude']=lattrain
df_train['Longitude']=lontrain

In [ ]:
df_train['coordinates'] = tuple(zip(df_train['Latitude'],df_train['Longitude']))

In [ ]:
df_train

### Compute Distance for each city between each airport and train station

In [ ]:
# Our distance function
from math import radians, cos, sin, asin, sqrt

def haversine(point1, point2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lat1, lon1=point1
    lat2, lon2=point2
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [ ]:
disonecity=[]
cityname=[]
aptname=[]
for coords,airport in zip(df_bigairports.coordinates, df_bigairports['Nom aéroport']):
    for coords2,city in zip(df_cities['coordinates'], df_cities.code_commune_INSEE):
        disonecity.append(haversine(coords,coords2))
        cityname.append(city)
        aptname.append(airport)

In [ ]:
d={'City':cityname,'Airport':aptname,'Distance':disonecity}
df_cityairport=pd.DataFrame(d)
df_cityairport.dropna(subset=['Distance'],inplace=True)

In [ ]:
df_distance=df_cityairport[['City','Airport','Distance']].groupby(['City'])['Distance'].min().to_frame()

In [ ]:
df_distance=df_distance.merge(df_cityairport,on='Distance',how='left')

In [ ]:
df_distance.rename(columns={'City':'code_commune_INSEE'},inplace=True)

In [ ]:
df_distance

In [ ]:
disonecity2=[]
cityname2=[]
aptname2=[]
for coords,train in zip(df_train.coordinates, df_train['UT']):
    for coords2,city in zip(df_cities['coordinates'], df_cities.code_commune_INSEE):
        disonecity2.append(haversine(coords,coords2))
        cityname2.append(city)
        aptname2.append(train)

In [ ]:
d2={'City':cityname2,'Train':aptname2,'Distance':disonecity2}
df_citytrain=pd.DataFrame(d2)
df_citytrain.dropna(subset=['Distance'],inplace=True)

In [ ]:
df_distance2=df_citytrain[['City','Train','Distance']].groupby(['City'])['Distance'].min().to_frame()

In [ ]:
df_distance2=df_distance2.merge(df_citytrain,on='Distance',how='left')

In [ ]:
df_distance2.rename(columns={'City':'code_commune_INSEE'},inplace=True)

In [ ]:
df_distance2=df_distance2.drop_duplicates()
df_distance2

### Merge to main databse

In [ ]:
df_cities.dropna(subset=['nom_departement'],inplace=True)
df_cities.reset_index(inplace=True)

In [ ]:
df_cities=df_cities.merge(df_distance,on='code_commune_INSEE',how='left')

In [ ]:
df_cities=df_cities.merge(df_distance2,on='code_commune_INSEE',how='left')
df_cities

### Add of other data (population, price per sqmeter, availability of public subsidies, biggest sectors in the city, index of quality of life)

In [ ]:
pop=pd.read_excel('../raw_data/pop.xls')
pop.rename(columns={'CODGEO':'code_commune_INSEE'},inplace=True)
pop=pop[['code_commune_INSEE','Population']]

In [ ]:
for x in range(len(df_cities['code_commune_INSEE'])):
    if len(df_cities['code_commune_INSEE'][x])<5:
        y='0'+str(df_cities['code_commune_INSEE'][x])
        df_cities['code_commune_INSEE'].iloc[x]=y

In [ ]:
df_cities=df_cities.merge(pop,on='code_commune_INSEE',how='left')

In [ ]:
#We filter the cities keeping the ones only below 30000 inhabitants ! Core of our topic !
df_cities=df_cities[df_cities['Population']<30000]
df_cities

In [ ]:
subsidies=pd.read_csv('../raw_data/aides+prixm2.csv')

In [ ]:
for x in range(len(subsidies['code_commune_INSEE'])):
    if len(subsidies['code_commune_INSEE'][x])<5:
        y='0'+str(subsidies['code_commune_INSEE'][x])
        subsidies['code_commune_INSEE'].iloc[x]=y

In [ ]:
df_cities=df_cities.merge(subsidies,on='code_commune_INSEE',how='left')
df_cities

In [ ]:
sectors=pd.read_csv('../raw_data/sectors.csv')
df_cities=df_cities.merge(sectors,on='code_commune_INSEE',how='left')
df_cities

In [ ]:
df_cities.drop(columns=['index','Unnamed: 0_x','Unnamed: 0_y'],inplace=True)

In [ ]:
df_cities['statut'] = df_cities['statut'].fillna(0)

In [ ]:
#Here we keep only cities from "Metropole (Main land)", not enough data about other territories
df_cities=df_cities.loc[df_cities['nom_region'].isin(['Auvergne-Rhône-Alpes', 'Hauts-de-France',"Provence-Alpes-Côte d'Azur", 'Grand Est', 'Occitanie','Normandie', 'Nouvelle-Aquitaine', 'Centre-Val de Loire','Bourgogne-Franche-Comté', 'Bretagne', 'Corse', 'Pays de la Loire','Île-de-France'])]


In [ ]:
df_cities.drop_duplicates(inplace=True)

In [ ]:
df_cities = df_cities.astype({"code_postal": str}, errors='raise') 

In [ ]:
df_cities = df_cities.astype({"coordinates": str}, errors='raise') 

In [ ]:
qlty=pd.read_csv('../raw_data/quality_doc4.csv')

In [ ]:
df_updated=df_cities.merge(qlty,how='left',on='code_commune_INSEE')
df_updated.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
df_updated.rename(columns={'global':'lifelevelscore'},inplace=True)
df_updated

### We add the closeness to sea and mountain for each city (according to their home department)

In [ ]:
df_updated['mountain_city']=0
for x in range(len(df_updated['code_commune_INSEE'])):
    if df_updated['code_departement'][x] in ['73','74','38','5','4','6','39','66','9','65','64','31','26','11','2A','2B']:
        df_updated['mountain_city'][x]=1
    else:
        df_updated['mountain_city'][x]=0

In [ ]:
df_updated['sea_city']=0
for x in range(len(df_updated['code_commune_INSEE'])):
    if df_updated['code_departement'][x] in ['2A','2B','6','83','13','30','34','11','66','64','40','33','17','85','44','56','29','22','35','50','14','76','80','62','59']:
        df_updated['sea_city'][x]=1
    else:
        df_updated['sea_city'][x]=0

### Elections Results for 2nd round 2022

In [ ]:
elections=pd.read_excel('../raw_data/resultats-par-niveau-subcom-t2-france-entiere.xlsx')

In [ ]:
elections['winner']=0
elections['winner_percentage']=0
for x in range(len(elections['Libellé de la commune'])):
    if elections['% Voix/Exp'][x]>elections['Unnamed: 32'][x]:
        elections['winner'][x]=elections['Nom'][x]+ ' ' +elections['Prénom'][x]
        elections['winner_percentage'][x]=elections['% Voix/Exp'][x]
    if elections['% Voix/Exp'][x]<elections['Unnamed: 32'][x]:
        elections['winner'][x]=elections['Unnamed: 28'][x] + ' ' +elections['Unnamed: 29'][x]
        elections['winner_percentage'][x]=elections['Unnamed: 32'][x]

In [ ]:
elections_short=elections[['Libellé de la commune','winner','winner_percentage']]
elections_short.rename(columns={'Libellé de la commune':'nom_commune_complet'},inplace=True)

In [ ]:
elections_short.drop_duplicates(subset='nom_commune_complet',keep='first',inplace=True)

In [ ]:
df_updated=df_updated.merge(elections_short,on='nom_commune_complet',how='left')
df_updated

### Check for NaN values and solution

In [ ]:
import numpy as np
df_updated[['PrixMoyen_M2','lifelevelscore','s1','s2','s3','s4','s5','winner','winner_percentage']]=df_updated[['PrixMoyen_M2','lifelevelscore','s1','s2','s3','s4','s5','winner','winner_percentage']].replace(0, np.nan)

In [ ]:
nan=df_updated[df_updated[['PrixMoyen_M2','lifelevelscore','winner','s1','s2','s3','s4','s5']].values==0]
nan.drop_duplicates(inplace=True)
nan

In [ ]:
nan2=df_updated[df_updated[['Population','PrixMoyen_M2','lifelevelscore','s1','s2','s3','s4','s5','winner']].isna().values]
nan2.drop_duplicates(inplace=True)
nan2

In [ ]:
#We add data from 2017 for missing sqmeters prices
M2_2017=pd.read_csv('../raw_data/prixm2-communes-2017.csv')
M2_2017.rename(columns={'INSEE_COM':'code_commune_INSEE'},inplace=True)
M2_2017=M2_2017[['code_commune_INSEE','Prixm2']]

In [ ]:
df_updated=df_updated.merge(M2_2017,how='left',on='code_commune_INSEE')
df_updated.PrixMoyen_M2.fillna(df_updated.Prixm2, inplace=True)
df_updated

In [ ]:
nan3=df_updated[df_updated['PrixMoyen_M2'].isna().values]
nan3.drop_duplicates(inplace=True)
nan3

In [ ]:
#We replace values still missing by the mean in the home department
dep=df_updated.groupby('code_departement')['PrixMoyen_M2'].mean().to_frame()
dep.rename(columns={'PrixMoyen_M2':'PrixMoyen_M2_dep'},inplace=True)

In [ ]:
df_updated=df_updated.merge(dep,how='left',on='code_departement')
df_updated.PrixMoyen_M2.fillna(df_updated.PrixMoyen_M2_dep, inplace=True)
df_updated

In [ ]:
nan4=df_updated[df_updated[['Population','PrixMoyen_M2','lifelevelscore','s1','s2','s3','s4','s5','winner']].isna().values]
nan4.drop_duplicates(inplace=True)
nan4

In [ ]:
df_updated.drop(columns=['Prixm2','PrixMoyen_M2_dep'],inplace=True)

In [ ]:
df_updated['code_commune_INSEE'].duplicated().any()

In [ ]:
df_updated

### Finally add the cluster number corresponding to our KMeans Clustering Analysis

In [ ]:
df_cluster=pd.read_csv('../raw_data/clusters.csv')

In [ ]:
df_updated=df_updated.merge(df_cluster, on='code_commune_INSEE',how='left')

In [ ]:
df_updated.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
df_updated['cluster']=df_updated['cluster'].astype(dtype='float64')
df_updated

### Export and upload to BigQuery

In [ ]:
import json
import os
from google.oauth2 import service_account
from google.cloud import bigquery
pathjson="../../hiddenpath.json"
CREDENTIAL_KEY = os.getenv('CREDENTIAL_KEY')
credentials = service_account.Credentials.from_service_account_file(pathjson)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [ ]:
import pandas_gbq
pandas_gbq.to_gbq(df_updated, 'hiddeninfo', project_id='hiddeninfo',credentials=credentials,if_exists='replace')